In [6]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.sklearn

In [7]:
!pip install boto3
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import boto3
import os

import mlflow



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Variables de entorno para MinIO
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'minioadmin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minioadmin'
os.environ['MLFLOW_ARTIFACT_ROOT'] = 'mlflow'


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Modelos a probar
models = [
    ('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=5)),
    ('SVC', SVC(kernel='linear')),
    ('GaussianNB', GaussianNB()),
    ('MLPClassifier', MLPClassifier(max_iter=1000))  # Ahora MLPClassifier está importado
]

# Cargar y preparar los datos
df_new = pd.read_csv('penguins_size.csv')
df_new.dropna(inplace=True)
df_new = pd.get_dummies(df_new, columns=['island', 'sex'], drop_first=True)

X_new = df_new.drop('species', axis=1)
y_new = df_new['species']
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# Para realizar un seguimiento de la mejor precisión y el modelo ganador
best_accuracy = 0
best_model = None
best_model_name = ""

# Conectar a MLflow
mlflow.set_tracking_uri("http://mlflow:5000")


# Función para ejecutar los experimentos
def run_experiment(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        print(f"Iniciando experimentación con el modelo {model_name}")
        
        # Entrenamiento del modelo
        model.fit(X_train, y_train)

        # Predicción
        y_pred = model.predict(X_test)

        # Calcular la precisión
        accuracy = accuracy_score(y_test, y_pred)

        # Registrar el modelo en MLflow
        mlflow.sklearn.log_model(model, model_name)

        # Registrar las métricas
        mlflow.log_metric("accuracy", accuracy)

        # Registrar los parámetros
        if model_name == 'KNeighborsClassifier':
            mlflow.log_param("n_neighbors", model.n_neighbors)
        elif model_name == 'SVC':
            mlflow.log_param("kernel", model.kernel)
        elif model_name == 'MLPClassifier':
            mlflow.log_param("max_iter", model.max_iter)

        print(f"Modelo {model_name} - Accuracy: {accuracy}")
        print(f"Modelo {model_name} registrado en MLflow y almacenado en el bucket S3")
        print(f"Run ID: {mlflow.active_run().info.run_id}")

        # Verificar si este modelo es el mejor
        global best_accuracy, best_model, best_model_name
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_model_name = model_name


run_experiment('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=5), X_train_new, X_test_new, y_train_new, y_test_new)

KeyboardInterrupt: 